# Семинар 9: функции и сортировка

### Функции

Функцией будем считать некоторый обособленный кусок кода, который можно вызвать из любой другой части кода.

In [10]:
def has_negative_number(list_numbers):
    for num in list_numbers:
        if num < 0:
            # ключевое слово return сразу говорит питону выйти из функции и вернуть значение
            return True

    return False  # вопрос: что вернет функция, если я забуду здесь return False написать?

In [ ]:
print(has_negative_number([1, 5, 2, 3]))

In [ ]:
def increment(lst):
    for i in range(len(lst)):
        lst[i] += 1

l = [1, 2, 3, 4]
increment(l)
print(l)

In [13]:
# тайпинги все еще сохраняют динамическую типизацию,
# но позволяют улучшить читаемость кода
# чекеры вроде mypy умеют анализировать код и проверять их корректность

from typing import Iterable

def has_negative_number(numbers: Iterable[int]) -> bool:
    for num in numbers:
        if num < 0:
            return True

    return False

In [ ]:
with open("numbers.txt", "r") as fin:
    while (line := fin.readline()):
        numbers = list(map(int, line.split()))
        print(f"{numbers} -> {has_negative_number(numbers)}")

Функцию можно сделать чуть более удобной для использования не только списком:

In [ ]:
# все аргументы через * типизируются без List
# https://peps.python.org/pep-0484/#arbitrary-argument-lists-and-default-argument-values

def has_negative_number(*numbers: int) -> bool:
    for num in numbers:
        if num < 0:
            return True

    return False

print(has_negative_number(1, 2, 3, 4, -1))  # теперь можно не передавать список

Помимо `*args` есть еще и `**kwargs`:

In [27]:
# на самом деле, в качестве тайпинга для values подошло бы
# какое-нибудь самописное comparable, но мы тут не будем с этим заморачиваться
# https://stackoverflow.com/questions/37669222/how-can-i-hint-that-a-type-is-comparable-with-typing


def max_value(*values: int | str, **params: bool) -> int | str | None:
    return_idx = params.get("return_idx", False)  # достаем позиционный аргумент

    if not values:  # если список пустой, вернем None
        return None

    max_value = values[0]
    max_value_idx = 0
    for i, value in enumerate(values):
        if value > max_value:
            max_value = value
            max_value_idx = i

    if return_idx:
        return max_value_idx
    return max_value

In [ ]:
print("max value:", max_value(6, -1, 2, 9, 1))  # сам максимум
print("max value index:", max_value(6, -1, 2, 9, 1, return_idx=True)) # индекс максимума

In [39]:
# на самом деле, в качестве тайпинга для values подошло бы
# какое-нибудь самописное comparable, но мы тут не будем с этим заморачиваться
# https://stackoverflow.com/questions/37669222/how-can-i-hint-that-a-type-is-comparable-with-typing


def max_value(*values: int | str, return_idx: bool = False) -> int | str | None:
    if not values:  # если список пустой, вернем None
        return None

    max_value = values[0]
    max_value_idx = 0
    for i, value in enumerate(values):
        if value > max_value:
            max_value = value
            max_value_idx = i

    if return_idx:
        return max_value_idx
    return max_value

In [ ]:
print("max value:", max_value(6, -1, 2, 9, 1))  # сам максимум
print("max value index:", max_value(6, -1, 2, 9, 1, return_idx=True)) # индекс максимума

In [52]:
def my_max(a, b):
    if a > b:
        return a
    return b

In [ ]:
my_max(b=-1, a=1)

### Рестрикшены

В аргументы функции можно написать пустую `*`. Тогда будет считаться, что все после этой звездочки -- это именованные аргументы и больше позиционные использовать нельзя. Ранее мы могли вызвать функцию вот так:

In [ ]:
def my_max(a, b, print_hello=True, **kwargs):
    if print_hello:
        print(a, b, **kwargs)

    if a > b:
        return a
    return b


my_max(1, 2, print_hello=True)

А если добавить, то уже нельзя:

In [61]:
def my_max(a, b, *, print_hello=True, **kwargs):
    if print_hello:
        print(a, b, **kwargs)

    if a > b:
        return a
    return b

In [ ]:
my_max(1, 2, print_hello=True)

In [ ]:
my_max(1, 2, True, sep='.', end='!')  # ошибка

In [ ]:
my_max(1, 2, print_hello=True, sep='.', end='!')  # все ОК

In [ ]:
my_max(a=1, b=2, print_hello=True, sep='.', end='!')  # и так тоже работать будет

Можно еще лучше: давайте запретим первым двум аргументам быть именованными в принципе. Это делается через `/`:

In [68]:
def my_max(a, b, /, c, *, print_hello=True, **kwargs):
    if print_hello:
        print(a, b, c, **kwargs)

    if a > b:
        return a
    return b

In [ ]:
my_max(a=1, b=2, c=3, print_hello=True, sep='.', end='!')  # ошибка

In [ ]:
my_max(1, 2, 3, print_hello=True, sep='.', end='!')  # все работает

In [ ]:
my_max(1, 2, c=3, print_hello=True, sep='.', end='!')  # и так тоже работает

### Области видимости

По умолчанию, все переменные в питоне, которые объявлены внутри функции, локальные. Вот парочка игрушечных примеров

In [ ]:
def f(x):
    square_x = x ** 2
    print(square_x)

f(10)

print(square_x)  # ну конечно, вам скорее всего даже редактор подчеркнет, что так низя

In [ ]:
value = 10

def f():
    print(value ** 2)  # но читать переменные из глобальной области можно

f()

In [ ]:
value = 10

def f():
    value += 1  # но примитивы менять низя (вы зачем вообще это делать хотите???)

f()

In [ ]:
value = 10

def f():
    global value  # глобал -- это очень и очень плохо
    # иногда полезно делать global для каких-то совсем важных штук, но это редкость
    value += 1  # но если очень хочется, то все-таки можно

f()

print(value)

In [ ]:
x = 6

def f():
    x = 1

    def g():
        nonlocal x
        x += 1
        print(x)

    return g

g = f()
g()
g()
g()

In [ ]:
credentials = None

def create_user():
    global credentials  # изначально None и поэтому просто так не обратимся

    credentials = {
        "login": "destroyer_2007",
        "password": "zavtra_v_shkolu((((",
    }

print(credentials)

create_user()

print(credentials)


In [ ]:
a = [1, 2, 3]  # a -----> [1, 2, 3, 4]

def f():
    a.append(4)  # но вы же не думали, что все так просто... подумайте, почему так происходит

f()

print(a)

In [ ]:
def f(a: list):  # равно так же как и поменяет список передача списка в функцию (ссылки же)
    a.append(4)

a = [1, 2, 3]

f(a)

print(a)

### Лямбды

Анонимные однострочные функции

In [ ]:
# вообще, присваивать лямбды переменным плохой тон

is_negative = lambda x: x < 0

is_negative(-1)

In [ ]:
next_value = lambda n: n + 1

print(next_value(1))

In [ ]:
(lambda *args: sum(args) / len(args))(5, 8, 7, 9)

In [ ]:
(lambda x, y: x + y)(5, 8)

In [ ]:
def square(x: int) -> int:
    return x ** 2


values = [1, 2, 3, 4, 5, 6]

print(list(map(square, values)))

In [ ]:
values = [1, 2, 3, 4, 5, 6]

print(list(map(lambda x: x ** 2, values)))

### Сортировка

Мы уже говорили про сортировку списков, но не говорили, что ей можно задать кастомный ключ сравнения через функции

In [ ]:
values = ["abcd", "aab", "bda", "0xabadbabe", "0xdeadbeef"]

sorted(values)  # все то же самое далее можно делать и с values.sort()

In [ ]:
values = ["abcd", "aab", "bda", "0xabadbabe", "0xdeadbeef"]

sorted(values, key=lambda s: len(s))  # все то же самое далее можно делать и с values.sort()

In [ ]:
values = ["abcd", "aab", "bda", "0xabadbabe", "0xdeadbeef"]

sorted(values, key=len)  # все то же самое далее можно делать и с values.sort()

А как нам сравнивать по нескольким значениям сразу? Например, есть числа, хотим отсортировать их по возрастанию длин, но при равенстве -- по убыванию самих чисел

In [ ]:
values = [7876510, 678, 456789, 789, 123456]

sorted(values, key=lambda num: (len(str(num)), -num))  # кортежи сравниваются поэлементно, поэтому это так и работает